In [1]:
import pandas as pd

In [2]:
tr = pd.read_csv('../1.기본/train_merge_basic.csv')
te = pd.read_csv('../1.기본/test_merge_basic.csv')

trade_count = pd.read_csv('../../preprocess/trade_count.csv')

# 파생변수 만들기
* 기본 데이터에 만들 수 있는 변수들을 붙인다.
* trade_count 또한 파생변수를 만들어 붙여준다.
* trade 데이터를 보면 동일 계정에게 반복하여 재화를 건내기만 한 계정이 존재함. -> 부정행위를 의심할 수 있다.

In [3]:
trade_count['trade_count'] = trade_count['rec_count'] + trade_count['give_count']

# 동일 계정으로부터 3회 이상 받은 계정 수
dup_rec = trade_count[trade_count['rec_count']>2].groupby('acc_id').agg({
    'acc_id': 'count',
    'rec_count': 'sum'
}).rename(columns={
    'acc_id': 'dup_rec_acc_count',
    'rec_count': 'dup_rec_count'
}).reset_index()

# 동일 계정에게 3회 이상 준 계정 수
dup_give = trade_count[trade_count['give_count']>2].groupby('acc_id').agg({
    'acc_id': 'count',
    'give_count': 'sum'
}).rename(columns={
    'acc_id': 'dup_give_acc_count',
    'give_count': 'dup_give_count'
}).reset_index()

In [4]:
to_trade = trade_count.groupby('acc_id').agg({
    'acc_id': 'count',
    'trade_count': 'sum'
}).rename(columns={
    'acc_id': 'trade_acc_count',
    'trade_count': 'total_trade_count'
}).reset_index()

to_trade = pd.merge(to_trade, pd.merge(dup_rec, dup_give, how='outer'), how='outer').fillna(0)
to_trade.head(10)

,acc_id,trade_acc_count,total_trade_count,dup_rec_acc_count,dup_rec_count,dup_give_acc_count,dup_give_count
0,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,2,18.0,1.0,5.0,1.0,10.0
1,0000548b3eecb5e8c6736f9408ddd58f49d0a553b6f1f8...,1,4.0,0.0,0.0,1.0,4.0
2,00006916672106f94f68d18e4a0532b53a40f4549d3a62...,2,9.0,0.0,0.0,1.0,4.0
3,0000d57170ee9d25dfa6cc8c7fbe88d2ffb561637020a4...,1,3.0,0.0,0.0,0.0,0.0
4,0001f130e89288ff0df167b998f6eb7085687f411fcb72...,1,1.0,0.0,0.0,0.0,0.0
5,00027152625b3c992d2e423b2d811d34c26a23eb3705d8...,1,8.0,0.0,0.0,1.0,8.0
6,000291b3c6e9c6012afed9dd99bbc834082622667fe3b7...,1,1.0,0.0,0.0,0.0,0.0
7,0002cb165b57f8ab2bea2e387a3bebcd1ee57b44229150...,35,50.0,1.0,3.0,1.0,3.0
8,00036e5b6a197c196fa10fc0ad4e2853b22294dff64d27...,2,59.0,0.0,0.0,2.0,59.0
9,00038d64fb50e6967fa4488c077614ba9509ed0869f0ad...,1,26.0,0.0,0.0,1.0,26.0


In [5]:
to = pd.concat([tr,te]).reset_index(drop=True)
to = pd.merge(to, to_trade, how='left').fillna(0)
to.head(20)

,wk,acc_id,cnt_dt,play_time,npc_exp,npc_hongmun,quest_exp,quest_hongmun,item_hongmun,game_combat_time,...,rec_costume_sum,give_costume_count,give_costume_sum,payment_amount,trade_acc_count,total_trade_count,dup_rec_acc_count,dup_rec_count,dup_give_acc_count,dup_give_count
0,7,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,5,5671.650000,173339,31434,10267620,16640780,0,1426.383333,...,0.0,0.0,0.0,0,2.0,18.0,1.0,5.0,1.0,10.0
1,8,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,7,8783.600000,238297,52097,17142400,34621610,0,2261.066667,...,0.0,0.0,0.0,0,2.0,18.0,1.0,5.0,1.0,10.0
2,1,0001f130e89288ff0df167b998f6eb7085687f411fcb72...,3,1.200000,0,0,0,0,0,0.000000,...,0.0,0.0,0.0,0,1.0,1.0,0.0,0.0,0.0,0.0
3,2,0001f130e89288ff0df167b998f6eb7085687f411fcb72...,3,1.366667,0,0,0,0,0,0.000000,...,0.0,0.0,0.0,0,1.0,1.0,0.0,0.0,0.0,0.0
4,3,0001f130e89288ff0df167b998f6eb7085687f411fcb72...,2,0.800000,0,0,0,0,0,0.000000,...,0.0,0.0,0.0,0,1.0,1.0,0.0,0.0,0.0,0.0
5,4,0001f130e89288ff0df167b998f6eb7085687f411fcb72...,4,2.666667,0,0,0,0,0,0.000000,...,0.0,0.0,0.0,0,1.0,1.0,0.0,0.0,0.0,0.0
6,5,0001f130e89288ff0df167b998f6eb7085687f411fcb72...,3,1.233333,0,0,0,0,0,0.000000,...,0.0,0.0,0.0,0,1.0,1.0,0.0,0.0,0.0,0.0
7,6,0001f130e89288ff0df167b998f6eb7085687f411fcb72...,4,2.400000,0,0,0,0,0,0.000000,...,0.0,0.0,0.0,0,1.0,1.0,0.0,0.0,0.0,0.0
8,7,0001f130e89288ff0df167b998f6eb7085687f411fcb72...,3,2.150000,0,0,0,0,0,0.000000,...,0.0,0.0,0.0,0,1.0,1.0,0.0,0.0,0.0,0.0
9,8,0001f130e89288ff0df167b998f6eb7085687f411fcb72...,3,1.283333,0,0,0,0,0,0.000000,...,0.0,0.0,0.0,0,1.0,1.0,0.0,0.0,0.0,0.0


# 파생변수 설명
* X1 : 플레이시간 대비 전투시간
* X2 : 플레이시간 대비 파티시간
* X3 : 전체 몹 경험치
* X4 : 전체 퀘스트 경험치
* X5 : 전체 홍문 경험치
* X6 : 전체 경험치
* X7 : 단위시간당 경험치
* X8 : 단위시간당 얻은 금화
* X9 : 전체 인던 입장 횟수
* X10 : 전체 레이드 입장 횟수
* X11 : 인던 + 레이드 입장 횟수
* X12 : 전체 인던 클리어 횟수
* X13 : 전체 레이드 클리어 횟수
* X14 : 인던 + 레이드 클리어 횟수
* X15 : 전체 채팅 횟수
* X16 : 일반 채팅 비율
* X17 : 귓속말 비율
* X18 : 지역 채팅 비율
* X19 : 파티 채팅 비율
* X20 : 길드 채팅 비율
* X21 : 세력 채팅 비율
* X22 : 단위시간당 채팅 횟수
* X23 : 주간 얻은 금화
* X24 : 거쳐간 금화 절대값
* X25 : 얻은 잡화
* X26 : 거쳐간 잡화 절대값
* X27 : 단위시간당 X23
* X28 : 단위시간당 X24
* X29 : 단위시간당 X25
* X30 : 단위시간당 X26
* X31 : 채집, 제작 횟수
* X32 : 단위시간당 X31
* X33 : 단위시간당 거래 횟수
* X34 : 단위시간당 유료결제량

In [6]:
to.columns

Index(['wk', 'acc_id', 'cnt_dt', 'play_time', 'npc_exp', 'npc_hongmun',
       'quest_exp', 'quest_hongmun', 'item_hongmun', 'game_combat_time',
       'get_money', 'duel_cnt', 'duel_win', 'partybattle_cnt',
       'partybattle_win', 'cnt_enter_inzone_solo', 'cnt_enter_inzone_light',
       'cnt_enter_inzone_skilled', 'cnt_enter_inzone_normal', 'cnt_enter_raid',
       'cnt_enter_raid_light', 'cnt_enter_bam', 'cnt_clear_inzone_solo',
       'cnt_clear_inzone_light', 'cnt_clear_inzone_skilled',
       'cnt_clear_inzone_normal', 'cnt_clear_raid', 'cnt_clear_raid_light',
       'cnt_clear_bam', 'normal_chat', 'whisper_chat', 'district_chat',
       'party_chat', 'guild_chat', 'faction_chat', 'cnt_use_buffitem',
       'gathering_cnt', 'making_cnt', 'party_time', 'party_count',
       'party_mem_count', 'mean_party_time', 'mean_party_mem',
       'total_guild_mem', 'guild_count', 'max_guild_mem', 'mean_guild_mem',
       'trade_count', 'rec_count', 'give_count', 'rec_money_count',
       '

In [7]:
to['X1'] = to['game_combat_time'] / to['play_time']
to['X2'] = to['party_time'] / to['play_time']
to['X3'] = to['npc_exp'] + to['npc_hongmun']
to['X4'] = to['quest_exp'] + to['quest_hongmun']
to['X5'] = to['npc_hongmun'] + to['quest_hongmun']
to['X6'] = to['X3'] + to['X4']
to['X7'] = to['X6'] / to['play_time']
to['X8'] = to['get_money'] / to['play_time']
to['X9'] = (to['cnt_enter_inzone_solo']
            + to['cnt_enter_inzone_light']
            + to['cnt_enter_inzone_skilled']
            + to['cnt_enter_inzone_normal'])
to['X10'] = to['cnt_enter_raid'] + to['cnt_enter_raid_light']
to['X11'] = to['X9'] + to['X10']
to['X12'] = (to['cnt_clear_inzone_solo']
            + to['cnt_clear_inzone_light']
            + to['cnt_clear_inzone_skilled']
            + to['cnt_clear_inzone_normal'])
to['X13'] = to['cnt_clear_raid'] + to['cnt_clear_raid_light']
to['X14'] = to['X12'] + to['X13']
to['X15'] = (to['normal_chat']
             + to['whisper_chat']
             + to['district_chat']
             + to['party_chat']
             + to['guild_chat']
             + to['faction_chat'])
to['X16'] = to['normal_chat'] / to['X15']
to['X17'] = to['whisper_chat'] / to['X15']
to['X18'] = to['district_chat'] / to['X15']
to['X19'] = to['party_chat'] / to['X15']
to['X20'] = to['guild_chat'] / to['X15']
to['X21'] = to['faction_chat'] / to['X15']
to['X22'] = to['X15'] / to['play_time']
to['X23'] = to['get_money'] + to['rec_money_sum'] - to['give_money_sum']
to['X24'] = to['get_money'] + to['rec_money_sum'] + to['give_money_sum']
to['X25'] = to['rec_grocery_sum'] - to['give_grocery_sum']
to['X26'] = to['rec_grocery_sum'] + to['give_grocery_sum']
to['X27'] = to['X23'] / to['play_time']
to['X28'] = to['X24'] / to['play_time']
to['X29'] = to['X25'] / to['play_time']
to['X30'] = to['X26'] / to['play_time']
to['X31'] = to['gathering_cnt'] + to['making_cnt']
to['X32'] = to['X31'] / to['play_time']
to['X33'] = to['trade_count'] / to['play_time']
to['X34'] = to['payment_amount'] / to['play_time']

In [8]:
to.head(20)

,wk,acc_id,cnt_dt,play_time,npc_exp,npc_hongmun,quest_exp,quest_hongmun,item_hongmun,game_combat_time,...,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34
0,7,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,5,5671.650000,173339,31434,10267620,16640780,0,1426.383333,...,-511.0,531.0,-0.034683,0.036005,-0.090097,0.093624,0,0.0,0.001058,0.0
1,8,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,7,8783.600000,238297,52097,17142400,34621610,0,2261.066667,...,-976.0,1016.0,-0.044944,0.046136,-0.111116,0.115670,0,0.0,0.001366,0.0
2,1,0001f130e89288ff0df167b998f6eb7085687f411fcb72...,3,1.200000,0,0,0,0,0,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0,0.0,0.000000,0.0
3,2,0001f130e89288ff0df167b998f6eb7085687f411fcb72...,3,1.366667,0,0,0,0,0,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0,0.0,0.000000,0.0
4,3,0001f130e89288ff0df167b998f6eb7085687f411fcb72...,2,0.800000,0,0,0,0,0,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0,0.0,0.000000,0.0
5,4,0001f130e89288ff0df167b998f6eb7085687f411fcb72...,4,2.666667,0,0,0,0,0,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0,0.0,0.000000,0.0
6,5,0001f130e89288ff0df167b998f6eb7085687f411fcb72...,3,1.233333,0,0,0,0,0,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0,0.0,0.000000,0.0
7,6,0001f130e89288ff0df167b998f6eb7085687f411fcb72...,4,2.400000,0,0,0,0,0,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0,0.0,0.000000,0.0
8,7,0001f130e89288ff0df167b998f6eb7085687f411fcb72...,3,2.150000,0,0,0,0,0,0.000000,...,0.0,0.0,-14000.000000,14000.000000,0.000000,0.000000,0,0.0,0.465116,0.0
9,8,0001f130e89288ff0df167b998f6eb7085687f411fcb72...,3,1.283333,0,0,0,0,0,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0,0.0,0.000000,0.0


In [9]:
new_tr = pd.merge(tr[['acc_id', 'wk']], to, how='left')
new_te = pd.merge(te[['acc_id', 'wk']], to, how='left')

In [10]:
new_tr.to_csv('train_create.csv', index=False)
new_te.to_csv('test_create.csv', index=False)